In [243]:
import ujson
import datetime
import os

def movie_aliases():
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)['movies']
        
    return list(map((lambda movie: movie['alias']),resource))


def get_movies_by_tag(tag):
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)['movies']
        
    return list(filter((lambda movie:  tag in movie['tag'] ), resource))

def get_movie_by_alias(alias):
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)['movies']
        
    return list(filter((lambda movie:  alias in movie['alias'] ), resource))

def get_movie_by_title(title):
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)['movies']
        
    return list(filter((lambda movie:  title in movie['title']), resource))


def movie_titles():
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)['movies']
        
    return list(map((lambda movie: movie['title']),resource))
    

def is_in_resource(resource, alias):
    
    if len(list(filter((lambda x : x['alias'] in alias), resource['movies']))) > 0:
        return(True)
    else:
        return(False)

def add(movie):
    
    if type(movie) is not dict: return("Movie need to be specified as key:value pairs in a dictionnary. Process Aborted.")
    
    if 'alias' not in movie.keys(): return "Update has no 'alias' key. Process Aborted."
    if 'tag' not in movie.keys(): return "Update has no 'tag' key. Process Aborted."
    if 'title' not in movie.keys(): return "Update has no 'title' key. Process Aborted."
    
    if 'movies_logs_resource.txt' not in os.listdir('.'):
            return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:  
        resource = ujson.load(json_file)
    
    if is_in_resource(resource, movie['alias']) == True :
        return "%s with alias '%s' and tag '%s' is already added. Need to update?.. use the update function" %(movie['title'], movie['alias'], movie['tag'])
    else:
        movie['timestamp'] = datetime.datetime.now()
        resource['movies'].append(movie)
        
        resource['logs'].append({
        'timestamp': datetime.datetime.now(),
        'type': 'post',
        'message': " '%s' with alias '%s' and tag '%s' was successfully added." %(movie['title'], movie['alias'], movie['tag'])
        })
        
        with open('movies_logs_resource.txt', 'w') as outfile:  
            ujson.dump(resource, outfile)
        
        return "%s with alias '%s' and tag '%s' was successfully added." %(movie['title'], movie['alias'], movie['tag'])


def delete(alias):
    if 'movies_logs_resource.txt' not in os.listdir('.'):
               return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:
               resource = ujson.load(json_file)
    
    if is_in_resource(resource, alias) == False :
        return "Movie with alias is not in resource file. Movie must be added first." %(alias)
    else:
        removing = list(filter((lambda movie : movie['alias'] in alias), resource['movies']))
        
        for movie in removing: resource['movies'].remove(movie)
        del removing
        
        with open('movies_logs_resource.txt', 'w') as outfile:  
            ujson.dump(resource, outfile)
        return True
        
    
def update(alias, updates):
    if type(update) is not dict: return("Updates need to be specified as key:value pairs in a dictionnary. Process Aborted.")
    keys = updates.keys()
    values = updates.values()
    
    if keys not in ['alias','tag','title']:
        return '''The updates' dictionary do not have the right keys; they must all be in ['alias','tag','title'].
        Note: Do not include 'timestamp' when doing updates. Process Aborted'''
    
    if len(keys) is not len(values):
        return("Number of Keys and Values do not match. Process Aborted.")
    
    def helper(movie, keys, values):
        for k in range(len(keys)):
            movie[keys[k]] = values[k]
        movie['timestamp'] = datetime.datetime.now()
        return movie
    
    if 'movies_logs_resource.txt' not in os.listdir('.'):
               return " The file 'movies_logs_resource.txt' is not in the current working directory. Process Aborted."
    
    with open('movies_logs_resource.txt') as json_file:
               resource = ujson.load(json_file)
    
    if is_in_resource(resource, alias) == False :
        return "Movie with alias is not in resource file. Movie must be added first." %(alias)
    else:
        movie = list(filter((lambda movie : movie['alias'] in alias), resource['movies']))
        if len(movie) is not 1 : return("That's weird...multiple matches for alias given. Process Aborted.")
        else: 
            updated = helper(movie[0], keys, values); del movie
            if 'alias' not in updated.keys(): return("Update has no 'alias' key. Process Aborted.")
            if 'tag' not in updated.keys(): return("Update has no 'tag' key. Process Aborted.")
            if 'title' not in updated.keys(): return("Update has no 'title' key. Process Aborted.")
            if 'timestamp' not in updated.keys(): return("Update has no 'timestamp' key. Process Aborted.")
            deleted = delete(alias)
            if deleted is not True : return deleted
            del deleted
            
            with open('movies_logs_resource.txt') as json_file:
                resource = ujson.load(json_file)
            
            resource['movies']. append(updated)
            resource['logs'].append({
                'timestamp': datetime.datetime.now(),
                'type': 'post',
                'message': " '%s' with alias '%s' and tag '%s' was successfully added as an update." %(updated['title'], updated['alias'], updated['tag'])
            
            })
            
            with open('movies_logs_resource.txt', 'w') as outfile:
                ujson.dump(resource, outfile)
            return " '%s' with alias '%s' and tag '%s' was successfully added as an update." %(updated['title'], updated['alias'], updated['tag'])
                       
               
        
    

In [207]:
with open('movies_logs_resource.txt') as json_file:
    resource = ujson.load(json_file)

In [208]:
test= list(filter((lambda movie : movie['alias'] in "test"), resource['movies']))[0]
test

{u'alias': u'test',
 u'tag': u'other',
 u'timestamp': 1494952445,
 u'title': u'Ahmed'}